In [3]:
using Random, Plots

function fitness(x)
    return 5 - 24x + 17x^2 - (11/3)x^3 + (1/4)x^4
end

function generate_population(size, bit_length)
    return [rand(0:(2^bit_length - 1)) for _ in 1:size]
end

function to_binary(x, bit_length)
    return lpad(string(x, base=2), bit_length, '0')
end

function crossover(parent1, parent2, crossover_point)
    bin1, bin2 = to_binary(parent1, 8), to_binary(parent2, 8)
    child1 = parse(Int, bin1[1:crossover_point] * bin2[crossover_point+1:end], base=2)
    child2 = parse(Int, bin2[1:crossover_point] * bin1[crossover_point+1:end], base=2)
    return child1, child2
end

function mutate(individual, mutation_rate, bit_length)
    bin_repr = to_binary(individual, bit_length)
    if rand() < mutation_rate
        mutation_point = rand(1:bit_length)
        new_bin = bin_repr[1:mutation_point-1] * (bin_repr[mutation_point] == '0' ? "1" : "0") * bin_repr[mutation_point+1:end]
        return parse(Int, new_bin, base=2)
    end
    return individual
end

function print_population(population, bit_length)
    println("ID | Десятичное | Двоичное  | Приспособленность")
    println("---|------------|-----------|------------------")
    for (i, x) in enumerate(population)
        println("$(i)  | $(x)        | $(to_binary(x, bit_length)) | $(fitness(x))")
    end
end

function visualize_generations(history, filename)
    x_vals = 0:0.1:10
    y_vals = fitness.(x_vals)
    anim = @animate for population in history
        plot(x_vals, y_vals, label="f(x)", lw=2)
        scatter!(population, fitness.(population), label="Популяция", color=:red)
    end
    Plots.gif(anim, filename, fps=2)
end

function genetic_algorithm(iterations, pop_size, bit_length, mutation_rate)
    population = generate_population(pop_size, bit_length)
    println("Начальная популяция:")
    print_population(population, bit_length)
    
    history = [copy(population)]
    for gen in 1:iterations
        sorted_population = sort(population, by=fitness)
        new_population = []
        for i in 1:2:(pop_size-1)
            p1, p2 = sorted_population[rand(1:pop_size)], sorted_population[rand(1:pop_size)]
            child1, child2 = crossover(p1, p2, rand(1:bit_length-1))
            push!(new_population, mutate(child1, mutation_rate, bit_length))
            push!(new_population, mutate(child2, mutation_rate, bit_length))
        end
        population = new_population
        push!(history, copy(population))
        println("\nПоколение $(gen):")
        print_population(population, bit_length)
    end
    
    visualize_generations(history, "genetic_algorithm.gif")
    return sort(population, by=fitness)[1]
end

bit_length = 4
best_solution = genetic_algorithm(100, 10, bit_length, 0.3)
println("\nЛучшее найденное решение: x = $best_solution, f(x) = $(fitness(best_solution))")

Начальная популяция:
ID | Десятичное | Двоичное  | Приспособленность
---|------------|-----------|------------------
1  | 2        | 0010 | -0.33333333333333215
2  | 13        | 1101 | 1650.583333333334
3  | 1        | 0001 | -5.416666666666666
4  | 7        | 0111 | 12.583333333333485
5  | 6        | 0110 | 5.0
6  | 5        | 0101 | 7.916666666666686
7  | 11        | 1011 | 577.916666666667
8  | 8        | 1000 | 47.66666666666674
9  | 1        | 0001 | -5.416666666666666
10  | 10        | 1010 | 298.3333333333335

Поколение 1:
ID | Десятичное | Двоичное  | Приспособленность
---|------------|-----------|------------------
1  | 13        | 1101 | 1650.583333333334
2  | 7        | 0111 | 12.583333333333485
3  | 13        | 1101 | 1650.583333333334
4  | 10        | 1010 | 298.3333333333335
5  | 13        | 1101 | 1650.583333333334
6  | 1        | 0001 | -5.416666666666666
7  | 10        | 1010 | 298.3333333333335
8  | 9        | 1001 | 133.25
9  | 9        | 1001 | 133.25
10  | 5       

┌ Info: Saved animation to c:\Users\Golum\Desktop\iu9-education\optimization-methods\let5\genetic_algorithm.gif
└ @ Plots C:\Users\Golum\.julia\packages\Plots\kLeqV\src\animation.jl:156
